<a href="https://www.kaggle.com/code/dazhengzhu/nlp-with-disaster-tweets?scriptVersionId=194661528" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Natural Language Processing with Disaster Tweets 

## Introduction

This project is to build a machine learning model that predicts which Tweets are about real disasters and which one’s aren’t. A dataset of 10,000 tweets that were hand classified is available.

Before we train a model on text data, we need to preprocess the text. In many cases, text needs to be **tokenized** and **vectorized** before it can be fed to a model, and in some cases the text requires additional preprocessing steps such as **normalization** and **feature selection**. After text is processed into a suitable format, we use it in natural language processing (NLP) workflow.

In this project we build two models for comparision: 
- Baseline model: The recurrent neural networks (RNN) are a class of neural networks that is powerful for modeling sequence data such as time series or natural language.
- SOTA model: The KerasNLP is a natural language processing library that supports workflows built from modular components that have state-of-the-art preset weights and architectures.

Finally, according to the performance of the model, we select the best model to make prediction.

### Key Takeaways

KerasNLP.DistilBERT model outperforms RNN model for disaster tweet classification, the findings include:

* **Higher Accuracy:** DistilBERT achieved a higher accuracy rate, indicating that it was more effective in correctly classifying disaster tweets.
* **Lower Loss:** The DistilBERT model exhibited a lower loss value, suggesting that it made fewer errors during training and testing.
* **Longer Training Time:** While DistilBERT required more time to train, its improved performance justified the increased computational cost.

### Setup

Set up Keras environment:

In [ ]:
# This sample uses Keras Core, the multi-backend version of Keras.
# The selected backend is TensorFlow (other supported backends are 'jax' and 'torch')
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

# !pip install keras_core
# !pip install keras_nlp

Import Tensorflow, Keras and other necessary libearies:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import string
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

### Data

#### Preview the dataset

In [ ]:
TRAINING_FILE_PATH = '../input/nlp-getting-started/train.csv'
TESTING_FILE_PATH = '../input/nlp-getting-started/test.csv'

df_train = pd.read_csv(TRAINING_FILE_PATH)
print(f'Training dataset:')
train_shape = df_train.shape
train_num_rows = train_shape[0]
print(f'Shape = {train_shape}')
print('Memory usage = {:.2f} MB'.format(df_train.memory_usage().sum() / 1024**2))
print('Head of data table:')
df_train.head()

Let's have a look at the train dataset.

- id: Tweet id in the table.
- keyword: A keyword from that tweet (may be blank)
- location: The location the tweet was sent from (may be blank)
- text: The text content of a tweet
- target: 1 if the tweet is a real disaster or 0 if not

In [ ]:
df_test = pd.read_csv(TESTING_FILE_PATH)
print(f'Testing dataset:')
test_shape = df_test.shape
# Save test rows for inference
test_num_rows = test_shape[0]
print(f'Shape = {test_shape}')
print('Memory usage = {:.2f} MB'.format(df_test.memory_usage().sum() / 1024**2))
print('Head of data table:')
df_test.head()

We can see there are 7613 records for training, the `text` column contains the tweet content, and the corresponding label in the `target` column. Also, there're 3263 tweets for testing.

### Load dataset

We'll use the `tf.data.experimental.make_csv_dataset` to load the csv data, and shuffle the data and create batches of these data. Note that we set `num_epochs=1`, this ensure the dataset only goes through the data once, which making it finite, if this property was not been set, **the `vectorizer.adapt` process will be blocked.**

Then we use given validation split ratio `VALIDATION_SPILT` to split the main dataset to testing & validation subset.

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.2

main_ds = tf.data.experimental.make_csv_dataset(
    TRAINING_FILE_PATH,
    select_columns=['text','target'],
    label_name='target',
    batch_size=BATCH_SIZE,
    num_epochs=1, # Ensure go through the data once to create a finite dataset
    shuffle=True,
    shuffle_buffer_size=BUFFER_SIZE,
    prefetch_buffer_size=tf.data.AUTOTUNE,
    num_rows_for_inference=train_num_rows # Help tf optimize the dataset creation
)

# Spilt the main dataset to train & validation datasets
num_val_elements = int(train_num_rows * VALIDATION_SPLIT // BATCH_SIZE)
raw_val_ds = main_ds.take(num_val_elements)
raw_train_ds = main_ds.skip(num_val_elements)

## Exploratory data analysis

### Preview the dataset spec

In [ ]:
raw_train_ds.element_spec

We can see each element is a `(dict, label)` tuple, and the `dict` contains `text` data.

### Preview tweets

Let's preview first 3 tweets, notice the data order is shuffled:

In [ ]:
for batch, label in raw_train_ds.take(1):
    for key, value in batch.items():
        print(f"{key:20s}: {value[:3]}")
    print()
    print(f"{'label':20s}: {label[:3]}")

### Analysis Tweets

Before analysis tweets, we need to perform the text preprocessing, this step is to transform raw text data into a structured format suitable for analysis or modeling, by applying techniques like tokenization, stemming, stopwords removal, and normalization, we aim to facilitate analysis, reduce dimensionality, and enhance model performance.

In [ ]:
def load_stopwords():
  nltk.download('punkt')
  nltk.download('stopwords')

load_stopwords()

# Function to preprocess text
def preprocess_text(text):
  # Tokenization
  words = nltk.word_tokenize(text=text, language='english')

  # Remove stop words
  stop_words = set(stopwords.words('english'))
  words = [word for word in words if word not in stop_words]

  # Stemming
  stemmer = PorterStemmer()
  words = [stemmer.stem(word) for word in words]

  return ' '.join(words)

# Apply preprocessing to the 'clean_text' column
df_train['clean_text'] = df_train['text'].apply(preprocess_text)

#### Show tweets Word Cloud

In [ ]:
text = ' '.join(df_train['clean_text'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')

We could observe from the Word Could picture:

- High-frequency disaster words: words like "help", "death", "life", "people", "survive", "kill" are common used function words in English.
- Specific disaster: words like "flood", "drown", "fire", "storm", "bomb" are consider specific disaster.

#### Show tweets Word Frequency

In [ ]:
def get_word_counts(text_data):
  all_words = []
  for text in text_data:
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    all_words.extend(words) 
  word_counts = Counter(all_words)
  word_freq = pd.DataFrame({'word': list(word_counts.keys()), 'count': list(word_counts.values())})
  word_freq = word_freq.sort_values(by='count', ascending=False)
  return word_freq

word_freq = get_word_counts(df_train['clean_text'])
word_freq.head(20)

Based on above Word Frequency data, the observations are:

- Common function words: words likes "I", "s", "the", "nt", "A", "m" are common function words used in language.
- Disaster specifics: words like "fire", "burn", "kill", and "bomb" suggest a focus on disaster specific.

### Data Preprocessing

#### Convert to text-only dataset

As the first preprocessing step, we'll need to convert the data type of dataset element as follows:

In [ ]:
def dict_to_text(dict, label):
  return dict['text'], label

train_ds = raw_train_ds.map(dict_to_text)
val_ds = raw_val_ds.map(dict_to_text)

### Create the TextVectorization layer

We will create a `TextVectorization` layer, this layer will be used to standardize, tokennize, and vectorize our data. We also set the output_mode to int to create unique integer indices for each token.

In [ ]:
from tensorflow.keras import layers

MAX_FEATURES = 10000
SEQUENCE_LENGTH = 160

vectorizer = layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

Next, we'll call the `adapt` to fit the state of the preprocessing layer to the text-only dataset. This will cause the model to build an index of strings to integers.

In [ ]:
# Make text-only dataset before adapt
text_ds = train_ds.map(lambda text, label:text)
vectorizer.adapt(text_ds)

The `.adapt` method sets the layer's vocabulary. Here are the first 20 tokens. After the padding `''` and unknown `[UNK]` tokens are sorted by frequency:

In [ ]:
vocabulary = np.array(vectorizer.get_vocabulary())
vocabulary[:20]

Once the vocabulary is set, the layer can vectorize text into indices, let's create a function to verctorize text:

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorizer(text), label

In [ ]:
# retrieve a batch (of 64 texts and labels) from the dataset
texts, labels = next(iter(train_ds))
text1, label1 = texts[0], labels[0]
vectorized_text, label = vectorize_text(text1, label1)
print("Original tweet: ", text1.numpy())
print("Label: ", label1.numpy())
print("Vectorized tweet: ", vectorized_text.numpy())
print("Round-trip tweet: ", " ".join(vocabulary[vectorized_text.numpy()[0]]))

We can see the tensor of indices are 0-padded to the longest sequence in the batch.

## Model

### Build a Baseline Model

We'll build a recurrent neural network (RNN), the layers are stacked sequentially to build the classifier:

- The first layer is the `vectorizer`, which converts the text to a sequence of token indices.

- After the `vectorizer` is an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

    This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a `layers.Dense` layer.

- The RNN processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input on the next timestep.

    The `layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the final output.

    - The main advantage of a bidirectional RNN is that the signal from the beginning of the input doesn't need to be processed all the way through every timestep to affect the output.

    - The main disadvantage of a bidirectional RNN is that you can't efficiently stream predictions as words are being added to the end.

- After the RNN has converted the sequence to a single vector the two `layers.Dense` do some final processing, and convert from this vector representation to a single logit as the classification output.


In [ ]:
model = tf.keras.Sequential([
    vectorizer,
    layers.Embedding(
        input_dim=len(vectorizer.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    layers.Bidirectional(layers.LSTM(64,  return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

model.summary()

The embedding layer uses masking to handle the varying sequence-lengths. All the layers after the Embedding support masking:

In [ ]:
print([layer.supports_masking for layer in model.layers])

#### Loss function and optimizer

Since this is a binary classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use `losses.BinaryCrossentropy` loss function.

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

#### Train the model

In [ ]:
EPOCHS = 10
rnn_history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = EPOCHS
)

### Build a SOTA Model

We'll choose the DistilBERT model, this model learns a distilled (approximate) version of BERT, retaining 97% performance but using only half the number of parameters ([paper](https://arxiv.org/abs/1910.01108)). 

It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

Specifically, it doesn't have token-type embeddings, pooler and retains only half of the layers from Google's BERT.

#### Create preprocessor and classifier

Text inputs need to be transformed to numeric token ids and arranged in several Tensors before being input to BERT.

The BertClassifier model can be configured with a preprocessor layer, in which case it will automatically apply preprocessing to raw inputs during fit(), predict(), and evaluate(). This is done by default when creating the model with from_preset().

In [ ]:
# Load a DistilBERT model.
preset= "distil_bert_base_en_uncased"

# Use a shorter sequence length.
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset,
                                                                   sequence_length=SEQUENCE_LENGTH,
                                                                   name="preprocessor_4_tweets"
                                                                  )

# Pretrained classifier.
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor, 
                                                               num_classes=2)

classifier.summary()

#### Compile classifier

For the NLP classifier, we'll use `losses.SparseCategoricalCrossentropy` loss function.

In [ ]:
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics= ["accuracy"]
)

#### Train the classifier

In [ ]:
nlp_history = classifier.fit(train_ds,
                         epochs=10, 
                         validation_data=val_ds)

## Results and analysis

In this section, we'll evaluate above created models.

### Evaluate the baseline model

Let's see how the model performs.

In [ ]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

The baseline model achieves an accuracy of about 95%, this indicates a good result.

#### Preview learning curve

Before show the learning curve, let's take a look at the `rnn_history` object:

In [ ]:
rnn_history.history.keys()

Then let's preview the baseline model learning curve:

In [ ]:
def preview_learning_curve(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')

preview_learning_curve(rnn_history)

The training loss decreases with each epoch and the training accuracy increases with each epoch, the loss and accuracy of validation performs similarly, these are expected behaves.

### Evaluate the sota model

#### Preview learning curve



In [ ]:
preview_learning_curve(nlp_history)

The training & validation loss decreases with each epoch and the training & validation accuracy increases with each epoch, compare with the baseline model, the nlp model achieves higher accuracy, lower loss, which reflects its more advanced strength. We'll select this model to make the prediction.

### Submission

For each tweets in the test set, we predict if the given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.

The `submission.csv` file uses the following format:
`id,target`

#### Prepare the Test Dataset

In [ ]:
raw_test_ds = tf.data.experimental.make_csv_dataset(
    TESTING_FILE_PATH,
    select_columns=['id','text'],
    batch_size=BATCH_SIZE,
    num_epochs=1, # Ensure go through the data once to create a finite dataset
    shuffle=False, # No shuffle for testing data
    prefetch_buffer_size=tf.data.AUTOTUNE,
    num_rows_for_inference=test_num_rows # Help tf optimize the dataset creation
)

In [ ]:
test_ds = raw_test_ds.map(lambda batch: batch['text'])

#### Make prediction

Predict by RNN model

In [ ]:
# Comment out to speed up notebook execution
# rnn_predictions = model.predict(test_ds)
# rnn_predictions

Predict by NLP model

In [ ]:
nlp_predictions = classifier.predict(test_ds)
nlp_predictions

#### Convert predictions to labels

For the rnn model prediction, the results have single column array, and if prediction >= 0.0, the label is 1 else it's 0.

In [ ]:
# Comment out to speed up
# This line used for convert rnn model predictions
# rnn_labels = np.where(predictions >= 0, 1, 0)
# rnn_labels

For the nlp model prediction, the results have two columns, for each row, if the first column value is larger, the label is 0, else it's 1.

In [ ]:
nlp_labels = np.argmax(nlp_predictions, axis=1)
nlp_labels

#### Save submission

In [ ]:
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [ ]:
# This line
submission['target'] = nlp_labels

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)

## Conclusion

Based on the experimental results, DistilBERT (Transformer) consistently demonstrated superior performance compared to the RNN model in the disaster tweet classification task.

The superior performance of DistilBERT can be attributed to its pre-trained nature and ability to capture complex language patterns. DistilBERT was pre-trained on a massive amount of text data, allowing it to learn contextual relationships between words and phrases. This pre-training enables DistilBERT to extract meaningful features from disaster tweets, leading to more accurate classifications.

In conclusion, DistilBERT is a promising approach for disaster tweet classification tasks. Its ability to achieve higher accuracy and lower loss, despite the increased training time, makes it a valuable tool for natural language processing applications in crisis management and information retrieval.


## Reference

- [Recurrent neural network](https://aws.amazon.com/what-is/recurrent-neural-network)
- [Google: Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification)
- [Tensorflow: Working with RNNs](https://www.tensorflow.org/guide/keras/working_with_rnns)
- [Tensorflow: Text classification with an RNN](https://www.tensorflow.org/text/tutorials/text_classification_rnn)
- [Tensorflow: Compute Similarity Metrics](https://www.tensorflow.org/text/tutorials/text_similarity)
- [Tensorflow: Basic text classification with Keras](https://www.tensorflow.org/tutorials/keras/text_classification)
- [Tensorflow: Classify text with BERT](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)
- [DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter (NeurIPS 2019)](https://arxiv.org/abs/1910.01108)